In [78]:
import pandas as pd
from feature_engine.creation import CyclicalFeatures 
data = pd.read_csv('../../data//promice/preprocessed/daily/KAN_L.csv')
data = data.drop(['Unnamed: 0','Surface height from combined measurements'], axis=1)
#

In [79]:
data.index = pd.to_datetime(data['Datetime'], format='%Y-%m-%d')
data['Datetime'] = pd.to_datetime(data['Datetime'], format='%Y-%m-%d')
#display(data)
print(data.info())
data['Ablation'].value_counts()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5286 entries, 2008-09-01 00:00:00+00:00 to 2023-02-20 00:00:00+00:00
Data columns (total 20 columns):
 #   Column                                           Non-Null Count  Dtype              
---  ------                                           --------------  -----              
 0   stid                                             5286 non-null   object             
 1   Datetime                                         5286 non-null   datetime64[ns, UTC]
 2   Air pressure (upper boom)                        5283 non-null   float64            
 3   Air temperature (upper boom)                     5283 non-null   float64            
 4   Relative humidity (upper boom) - corrected       5283 non-null   float64            
 5   Specific humidity (upper boom)                   5283 non-null   float64            
 6   Wind speed (upper boom)                          5279 non-null   float64            
 7   Wind from direction (upper boo

False    3473
True     1813
Name: Ablation, dtype: int64

In [80]:
def add_features(data):
    df = data.copy()
    df['DayOfYear'] = df['Datetime'].dt.dayofyear
    df['WeekNum'] = df['Datetime'].dt.isocalendar().week.astype(int)
    df['Month'] = df['Datetime'].dt.month
    df['Year'] = df['Datetime'].dt.year
    return df
#df = add_features(data)

In [60]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.compose import make_column_transformer

def pre_process(df, show=False, cyclical = False):
    df = df.copy()
    cyclical_cols = ['DayOfYear','Month','WeekNum', 'Wind from direction (upper boom)']
    numerical_cols = [x for x in df.select_dtypes(include=['int64','float64']).drop('Surface height from combined measurements DELTA',axis=1).columns.to_list() if x not in cyclical_cols]

    df = df.ffill().dropna()

    if cyclical==True:
        cyclical = CyclicalFeatures(variables=cyclical_cols, max_values={'Month':12,'WeekNum':52,'DayOfYear':365,'Wind from direction (upper boom)':360},drop_original=True)
        cyclical_values = cyclical.fit_transform(df[cyclical_cols])
        df = df.drop(cyclical_cols,axis=1)

        df = df.merge(cyclical_values,how='left',left_index=True,right_index=True)
        cyclical_cols = cyclical_values.columns.to_list()

    num_transformer = make_pipeline(
        SimpleImputer(strategy='mean'),
        MinMaxScaler())

    preprocessor = make_column_transformer(
    (num_transformer, numerical_cols)
    )

    df[numerical_cols] = preprocessor.fit_transform(df)
    
    if show and cyclical:
        df_year = cyclical_values[cyclical_values.index.year == 2019]
        plot_cols = cyclical_values.columns.to_list()[:-2]
    
        #plotting cyclical_conversion
        df_year[plot_cols].plot()
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'Data for year 2019')
        plt.show()
    return df
#df = add_features(data)
#df = pre_process(df, show=True, cyclical = True)

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

data = df.copy()

# Define a function to train and evaluate each model
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    return mse

# Split the dataset into training and testing sets
X = data.drop('target_variable', axis=1)
y = data['target_variable']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ARIMA model
arima_model = ARIMA(y_train, order=(1, 1, 1))
arima_mse = evaluate_model(arima_model, X_train, X_test, y_train, y_test)
print(f"ARIMA Model MSE: {arima_mse}")

# SARIMA model
sarima_model = SARIMAX(y_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_mse = evaluate_model(sarima_model, X_train, X_test, y_train, y_test)
print(f"SARIMA Model MSE: {sarima_mse}")

# XGBoost model
xgb_model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1)
xgb_mse = evaluate_model(xgb_model, X_train, X_test, y_train, y_test)
print(f"XGBoost Model MSE: {xgb_mse}")

# SVM model
svm_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svm_mse = evaluate_model(svm_model, X_train, X_test, y_train, y_test)
print(f"SVM Model MSE: {svm_mse}")

# Choose the best model based on the lowest MSE
model_mse = {
    'ARIMA': arima_mse,
    'SARIMA': sarima_mse,
    'XGBoost': xgb_mse,
    'SVM': svm_mse
}

best_model_name = min(model_mse, key=model_mse.get)
best_model = {
    'ARIMA': arima_model,
    'SARIMA': sarima_model,
    'XGBoost': xgb_model,
    'SVM': svm_model
}[best_model_name]

print(f"Best Model: {best_model_name}")


# Predict the beginning of the ice melt date for each year in the dataset
unique_years = data['year'].unique()
melt_dates = {}

for year in unique_years:
    year_data = data[data['year'] == year]
    X_year = year_data.drop('target_variable', axis=1)
    
    # Use the best model to predict the beginning of the ice melt date for the current year
    melt_date_prediction = best_model.predict(X_year)
    
    # Store the predicted melt date for the current year
    melt_dates[year] = melt_date_prediction

# Print the predicted melt dates for each year
for year, melt_date in melt_dates.items():
    print(f"Predicted melt date for {year}: {melt_date}")



# Define a function to determine if a given day meets the ice melt criteria
def is_melt_day(row):
    return (row['Ablation'] == True)

# Initialize an empty list to store the first melt day of each year
first_melt_days = []

# Iterate through each unique year in the dataset
unique_years = data['Year'].unique()
for year in unique_years:
    # Filter data for the current year
    year_data = data[data['Year'] == year]

    # Find the first melt day of the year
    first_melt_day = year_data[year_data.apply(lambda row: is_melt_day(row), axis=1)].iloc[0]

    # Store the first melt day (date or day of the year) in the list
    first_melt_days.append(first_melt_day['DayOfYear'])

# Add the first melt day of the year as a target feature to the dataset
data['first_melt_day'] = first_melt_days

# Save the modified dataset with the target feature
data.to_csv('path_to_modified_dataset.csv', index=False)


# Define a function to determine if a given day meets the ice melt criteria
def is_melt_day(row):
    return (row['Ablation'] == True)

# Initialize an empty list to store the first melt day of each year
first_melt_days = []

# Iterate through each unique year in the dataset
unique_years = data['Year'].unique()
for year in unique_years:
    # Filter data for the current year
    year_data = data[data['Year'] == year]

    # Find the first melt day of the year
    first_melt_day = year_data[year_data.apply(lambda row: is_melt_day(row), axis=1)].iloc[0]

    # Store the first melt day (date or day of the year) in the list
    first_melt_days.append(first_melt_day['DayOfYear'])

# Add the first melt day of the year as a target feature to the dataset
data['first_melt_day'] = first_melt_days

In [84]:
df = add_features(data)

df = df[df['Ablation'] == True]
unique_years = df['Year'].unique()
dates = {}
print(unique_years)
for i in unique_years:
    year_data = df.loc[df['Year'] == i]
    #display(year_data['Datetime'])
    dates[i] = year_data['Datetime'].min()
display(dates)





[2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021
 2022]


{2008: Timestamp('2008-09-01 00:00:00+0000', tz='UTC'),
 2009: Timestamp('2009-05-20 00:00:00+0000', tz='UTC'),
 2010: Timestamp('2010-05-07 00:00:00+0000', tz='UTC'),
 2011: Timestamp('2011-05-13 00:00:00+0000', tz='UTC'),
 2012: Timestamp('2012-05-11 00:00:00+0000', tz='UTC'),
 2013: Timestamp('2013-05-27 00:00:00+0000', tz='UTC'),
 2014: Timestamp('2014-05-23 00:00:00+0000', tz='UTC'),
 2015: Timestamp('2015-05-09 00:00:00+0000', tz='UTC'),
 2016: Timestamp('2016-05-01 00:00:00+0000', tz='UTC'),
 2017: Timestamp('2017-05-09 00:00:00+0000', tz='UTC'),
 2018: Timestamp('2018-05-03 00:00:00+0000', tz='UTC'),
 2019: Timestamp('2019-05-02 00:00:00+0000', tz='UTC'),
 2020: Timestamp('2020-05-01 00:00:00+0000', tz='UTC'),
 2021: Timestamp('2021-05-01 00:00:00+0000', tz='UTC'),
 2022: Timestamp('2022-05-30 00:00:00+0000', tz='UTC')}

In [99]:
data = df.copy()
# Define a function to determine if a given day meets the ice melt criteria
def is_melt_day(row):
    return row['Ablation'] == True

# Initialize an empty list to store the first melt day of each year
first_melt_days = []
data['first_melt_day'] = data['DayOfYear'].copy()
# Iterate through each unique year in the dataset
unique_years = data['Year'].unique()
for year in unique_years:
    # Filter data for the current year
    year_data = data[data['Year'] == year]

    # Find the first melt day of the year
    first_melt_day = year_data[year_data.apply(lambda row: is_melt_day(row), axis=1)].iloc[0]['DayOfYear']
    print(first_melt_day)
    # Store the first melt day (date or day of the year) in the list
    data.loc[data['Year'] == year,['first_melt_day']] = first_melt_day
display(data)

# Add the first melt day of the year as a target feature to the dataset
#data['first_melt_day'] = np.where(df['Datetime'].isin(first_melt_days), )


245
140
127
133
132
147
143
129
122
129
123
122
122
121
150


,stid,Datetime,Air pressure (upper boom),Air temperature (upper boom),Relative humidity (upper boom) - corrected,Specific humidity (upper boom),Wind speed (upper boom),Wind from direction (upper boom),Downwelling shortwave radiation - corrected,Upwelling shortwave radiation - corrected,...,Surface height from combined measurements DELTA,Albedo,Cloud cover,Ablation,Melting Season,DayOfYear,WeekNum,Month,Year,first_melt_day
Datetime,,,,,,,,,,,,,,,,,,,,,
2008-09-01 00:00:00+00:00,KAN_L,2008-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.099,NaN,NaN,True,beginning,245,36,9,2008,245
2008-09-02 00:00:00+00:00,KAN_L,2008-09-02 00:00:00+00:00,930.272417,4.067042,80.716750,4.420708,4.616417,151.127125,79.998500,43.203958,...,-0.053,0.545000,0.684958,True,beginning,246,36,9,2008,245
2008-09-03 00:00:00+00:00,KAN_L,2008-09-03 00:00:00+00:00,928.642042,2.856625,84.547542,4.257667,2.589208,130.834917,99.346417,51.914583,...,-0.023,0.525000,0.808042,True,beginning,247,36,9,2008,245
2008-09-04 00:00:00+00:00,KAN_L,2008-09-04 00:00:00+00:00,927.952250,2.757833,75.053708,3.746167,3.055417,126.096500,167.023750,95.056417,...,-0.052,0.568125,0.507583,True,middle,248,36,9,2008,245
2008-09-05 00:00:00+00:00,KAN_L,2008-09-05 00:00:00+00:00,914.642583,2.768333,87.821500,4.468500,2.539667,127.975583,49.319542,28.411292,...,-0.002,0.576571,0.945250,True,middle,249,36,9,2008,245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26 00:00:00+00:00,KAN_L,2022-09-26 00:00:00+00:00,927.875000,4.839750,90.168750,5.214625,5.004958,114.150000,43.918583,24.271542,...,-0.046,0.546000,0.866542,True,end,269,39,9,2022,150
2022-09-27 00:00:00+00:00,KAN_L,2022-09-27 00:00:00+00:00,930.958333,2.949792,90.035000,4.544208,3.331750,112.820833,68.477000,34.898917,...,-0.031,0.502000,0.799667,True,end,270,39,9,2022,150
2022-09-28 00:00:00+00:00,KAN_L,2022-09-28 00:00:00+00:00,933.750000,1.648958,98.920833,4.545083,2.514667,72.427917,54.783458,27.246167,...,-0.026,0.497500,0.893000,True,end,271,39,9,2022,150
